<a href="https://colab.research.google.com/github/AbdelatifBous/CNN-from-Scratch-/blob/main/CNN_python_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://www.ece.rice.edu/~wakin/images/lenaTest2.jpg

--2021-03-11 00:22:15--  https://www.ece.rice.edu/~wakin/images/lenaTest2.jpg
Resolving www.ece.rice.edu (www.ece.rice.edu)... 128.42.123.20
Connecting to www.ece.rice.edu (www.ece.rice.edu)|128.42.123.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37926 (37K) [image/jpeg]
Saving to: ‘lenaTest2.jpg’

lenaTest2.jpg       100%[===================>]  37.04K  --.-KB/s    in 0.04s   

2021-03-11 00:22:16 (907 KB/s) - ‘lenaTest2.jpg’ saved [37926/37926]



In [ ]:
import numpy as np 
from numpy import asarray 
import cv2
import matplotlib.pyplot as plt 

In [ ]:
img = cv2.imread('lenaTest2.jpg', cv2.IMREAD_GRAYSCALE) /255
plt.imshow(img, cmap = 'gray')
plt.show()
img.shape

In [ ]:
class Conv_op:
  def __init__(self, num_filters, filter_size):
    self.num_filters = num_filters
    self.filter_size = filter_size
    self.conv_filter = np.random.randn(num_filters, filter_size, filter_size)/ (filter_size * filter_size)

  def image_region(self, image):
    height, width = image.shape
    self.image = image
    for j in range(height - self.filter_size + 1 ):
      for k in range(width - self.filter_size + 1): 
        image_patch = image[ j : (j + self.filter_size), k: (k + self.filter_size)]
        yield image_patch, j, k 
  
  def forward_prop(self, image):
    height, width = image.shape
    conv_out = np.zeros((height - self.filter_size + 1, width - self.filter_size + 1, self.num_filters))
    for image_patch, i, j in self.image_region(image):
      conv_out[i,j] = np.sum(image_patch*self.conv_filter, axis = (1,2))
    return conv_out

  def back_prop(self, dl_dout, learning_rate):
    dl_df_params = np.zeros(self.conv_filter.shape)
    for image_patch, i, j in self.image_region(self.image):
      for k in range(self.num_filters):
        dl_df_params[k] += image_patch*dl_dout[i,j,k]

      # filter params update 
    self.conv_filter -= learning_rate*dl_df_params
    return dl_df_params

In [ ]:
conn = Conv_op(18,7)
out = conn.forward_prop(img)
out.shape

(506, 506, 18)

In [ ]:
plt.imshow(out[:,:,17], cmap = 'gray') # we have 18 images
plt.show()

In [ ]:
class Max_Pool:
  def __init__(self, filter_size):
    self.filter_size = filter_size

  def image_region(self, image):
    new_height = image.shape[0] // self.filter_size
    new_width = image.shape[1] // self.filter_size
    self.image = image 

    for i in range(new_height):
      for j in range(new_width):
        image_patch = image[(i*self.filter_size) : (i*self.filter_size + self.filter_size) , (j*self.filter_size) : (j*self.filter_size + self.filter_size)]
        yield image_patch, i, j

  def forward_prop(self, image):
    height, width, num_filters = image.shape
    output = np.zeros((height // self.filter_size, width // self.filter_size, num_filters))

    for image_patch , i, j in self.image_region(image):
      output[i,j] = np.amax(image_patch, axis = (0,1))

    return output

  def back_prop(self, dl_dout):
    dl_dmax_pool = np.zeros(self.image.shape)
    for image_patch, i, j in self.image_region(self.image):
      height, width, num_filters = image_patch.shape
      maximum_val = np.amax(image_patch, axis = (0,1))

      for i1 in range(height):
        for j1 in range(width):
          for k1 in range(num_filters):
            if image_patch[i1, j1, k1] == maximum_val[k1]:
              dl_dmax_pool[i*self.filter_size + i1 , j*self.filter_size + j1, k1] = dl_dout[i, j, k1]

      return dl_dmax_pool

In [ ]:
conn2 = Max_Pool(4)
out2 = conn2.forward_prop(out)
out2.shape

(126, 126, 18)

In [ ]:
plt.imshow(out2[:,:,17], cmap = 'gray') # we have 18 images
plt.show()

In [ ]:
class Softmax:
  def __init__(self, input_node, softmax_node):
    self.weight = np.random.randn(input_node, softmax_node)/input_node
    self.bias = np.zeros(softmax_node)

  def forward_prop(self, image):

    self.orig_im_shape = image.shape # used in backprop 
    image_modified = image.flatten()
    self.modified_input = image_modified # to be used in backprop 
    output_val = np.dot(image_modified, self.weight) + self.bias
    self.out = output_val 
    exp_out = np.exp(output_val)
    return exp_out/np.sum(exp_out, axis = 0) 

  def back_prop(self, dl_dout, learning_rate):
  	for i, grad in enumerate(dl_dout):
  		if grad == 0:
  			continue

  	transformation_eq = np.exp(self.out)
  	S_total = np.sum(transformation_eq)

  	# Gradients with respect to out (z)
  	dy_dz = -transformation_eq[i]*transformation_eq / (S_total **2)
  	dy_dz[i] = transformation_eq[i]*(S_total - transformation_eq[i]) / (S_total **2)

  	# Gradiente of totals against weights/biases/input
  	dz_dw = self.modified_input
  	dz_db = 1
  	dz_d_inp = self.weight

  	# Gradients of loss against totals 
  	dl_dz = grad * dy_dz

  	# Gradients of loss against weights/biases/input
  	dl_dw = dz_dw[np.newaxis].T @ dl_dz[np.newaxis]
  	dl_db = dl_dz * dz_db
  	dl_d_inp = dz_d_inp @ dl_dz

  	# update weights and biases 
  	self.weight -= learning_rate * dl_dw
  	self.bias -= learning_rate * dl_db

  	return dl_d_inp.reshape(self.orig_im_shape)

In [ ]:
conn3 = Softmax(54*54*18, 10)
out3 = conn3.forward_prop(out2)
print(out3)

In [ ]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
train_images = X_train[:1500]
train_labels = y_train[:1500]
test_images = X_test[:1500]
test_labels = y_test[:1500]

In [ ]:
conv = Conv_op(8,3)                   # 28*28*1  ->  26*26*8
pool = Max_Pool(2)                    # 26*26*8  ->  13*13*8
softmax = Softmax(13 * 13 * 8, 10)    # 13*13*8  ->  10

In [ ]:
def cnn_forward_prop(image, label):

	out_p = conv.forward_prop((image / 255) - 0.5)
	out_p = pool.forward_prop(out_p)
	out_p = softmax.forward_prop(out_p)

	# Calculate cross-entropy loss and accuracy.
	cross_ent_loss = -np.log(out_p[label])
	accuracy_eval = 1 if np.argmax(out_p) == label else 0

	return out_p, cross_ent_loss, accuracy_eval


def training_cnn(image, label, learn_rate=0.005):

	# Forward 
	out, loss, acc = cnn_forward_prop(image, label)

	# Calculate initial gradinent
	gradinent = np.zeros(10)
	gradinent[label] = -1 / out[label]

	# Backprop 
	grad_back = softmax.back_prop(gradinent, learn_rate)
	grad_back = pool.back_prop(grad_back)
	grad_back = conv.back_prop(grad_back, learn_rate)

	return loss, acc

In [ ]:
for epoch1 in range(4):
	print('Epoch %d ----->' % (epoch1 + 1))

	# Shuffle the training data 
	shuffle_data = np.random.permutation(len(train_images))
	train_images = train_images[shuffle_data]
	train_labels = train_labels[shuffle_data]

	# Training the CNN
	loss = 0
	num_correct = 0 
	for i, (im, label) in enumerate(zip(train_images, train_labels)): 
		if i % 100 == 0:
			print('%d steps out of 100 steps: Average Loss %.3f and Accuracy: %d%%' %(i + 1, loss / 100, num_correct) )
			loss = 0 
			num_correct = 0 
		ll, accu = training_cnn(im, label)
		loss += ll 
		num_correct += accu

# Testing the CNN 
print('**Testing phase')
loss = 0 
num_correct = 0 
for im, label in zip(test_images, test_labels):
	_, ll, accu = cnn_forward_prop(im, label)
	loss += ll 
	num_correct += accu

num_tests = len(test_images)
print( 'Test Loss:', loss / num_tests)
print( 'Test Accuracy:', num_correct / num_tests)

Epoch 1 ----->
1 steps out of 100 steps: Average Loss 0.000 and Accuracy: 0%
101 steps out of 100 steps: Average Loss 2.330 and Accuracy: 3%
201 steps out of 100 steps: Average Loss 2.384 and Accuracy: 13%
301 steps out of 100 steps: Average Loss 2.798 and Accuracy: 11%
401 steps out of 100 steps: Average Loss 3.043 and Accuracy: 10%
501 steps out of 100 steps: Average Loss 3.231 and Accuracy: 13%
601 steps out of 100 steps: Average Loss 3.446 and Accuracy: 14%
701 steps out of 100 steps: Average Loss 3.708 and Accuracy: 11%
801 steps out of 100 steps: Average Loss 3.947 and Accuracy: 7%
901 steps out of 100 steps: Average Loss 4.112 and Accuracy: 6%
1001 steps out of 100 steps: Average Loss 3.988 and Accuracy: 12%
1101 steps out of 100 steps: Average Loss 4.087 and Accuracy: 12%
1201 steps out of 100 steps: Average Loss 4.179 and Accuracy: 13%
1301 steps out of 100 steps: Average Loss 4.624 and Accuracy: 5%
1401 steps out of 100 steps: Average Loss 4.192 and Accuracy: 16%
Epoch 2 ----